In [78]:
# !pip install numpy-financial

## Imports

In [79]:
import requests

# To get analysts info
import yahoo_fin.stock_info as si

# Required to convert to numeric
import pandas as pd

# Required to calculate net present value
import numpy_financial as npf

# For reading properties
from jproperties import Properties

## Load properties

In [80]:
configs = Properties()

with open('config/av_dcf.properties', 'rb') as config_file:
     configs.load(config_file)

TICKER = configs.get('TICKER').data
# Perpetual growth rates is read as a string; we need to convert it to float
PERP_GROWTH_RATE = float(configs.get('PERP_GROWTH_RATE').data)
WACC_RATE = float(configs.get('WACC_RATE').data)
API_KEY = configs.get('ALPHAV_API_KEY').data

## Constants

In [81]:
PROJECTION_YEARS = 5
BASE_URL = 'https://www.alphavantage.co/query?'

## Utility method to get data from AV

In [82]:
def get_response(function):
    return requests.get(f'{BASE_URL}function={function}&symbol={TICKER}&apikey={API_KEY}')

## Get Growth Rate

In [83]:
growth_est_df = si.get_analysts_info(TICKER)['Growth Estimates']
growth_str = growth_est_df[growth_est_df['Growth Estimates'] == 'Next 5 Years (per annum)'][TICKER].iloc[0]
growth_rate = round(float(growth_str.rstrip('%')) / 100.0, 4)
growth_rate

0.0636

## Read from Alpha Vantage

In [84]:
# Income response
latest_income_stmt = get_response('INCOME_STATEMENT').json()['annualReports'][0]

# Balance sheet response
latest_balance_sheet = get_response('BALANCE_SHEET').json()['annualReports'][0]

# Cash flow response
latest_cash_flow = get_response('CASH_FLOW').json()['annualReports'][0]

# Overview for Beta
overview = get_response('OVERVIEW').json()

## Future Free Cash Flow (FFCF)

In [85]:
# Free cash flow = operating cash flow - capital expenditures
free_cash_flow = pd.to_numeric(latest_cash_flow['operatingCashflow']) - pd.to_numeric(latest_cash_flow['capitalExpenditures'])

ffcf = []
# Year 1
ffcf.append(free_cash_flow * (1 + growth_rate))

# Starting from year 2
for i in range(1, PROJECTION_YEARS):
    ffcf.append(ffcf[i-1] * (1 + growth_rate))

[118530774800.00002,
 126069332077.28003,
 134087341597.39505,
 142615296522.98938,
 151685629381.85153]

## Calculate the Terminal Value

In [86]:
# forecast_fcf[-1] refers to the last year in the growth period
terminal_value = ffcf[-1] * (1 + PERP_GROWTH_RATE)/(WACC_RATE - PERP_GROWTH_RATE)

## Enterprise Valuation

In [87]:
# Add the terminal value to the last year
ffcf[-1] = ffcf[-1] + terminal_value

# Calculate the enterprise_value using npv - add zero or else the method assumes
# first value as the initial investment
enterprise_value = npf.npv(WACC_RATE, [0] + ffcf)
# enterprise_value = sum(pv_ffcf)

## Equity Valuation

In [88]:
# Calculate Cash And Cash Equivalents
cash_and_equivalents = pd.to_numeric(latest_balance_sheet['cashAndCashEquivalentsAtCarryingValue'])

# Current debt - only interested in the value
current_debt = pd.to_numeric(latest_balance_sheet['currentDebt'])

# Long term debt - only interested in the value
long_term_debt = pd.to_numeric(latest_balance_sheet['longTermDebt'])

# Equity value -> enterprise_value + cash - total debt
equity_value = enterprise_value + cash_and_equivalents - (current_debt +  long_term_debt)

## Intrinsic Valuation

In [89]:
# Shares outstanding
shares_outstanding = pd.to_numeric(overview['SharesOutstanding'])

# Current stock price
current_value = si.get_live_price(TICKER)

# Intrinsic value
intrinsic_value = equity_value / shares_outstanding
'Current Price: ${:.2f} Intrinsic Value: ${:.2f}'.format(round(current_value,2), round(intrinsic_value,2)) + ' for ticker ' + TICKER

'Current Price: $174.49 Intrinsic Value: $117.15 for ticker AAPL'

In [90]:
upside = (intrinsic_value / current_value) - 1
'{:.2%}'.format(round(upside,4)) + ' for ticker ' + TICKER

'-32.86% for ticker AAPL'

In [91]:
if intrinsic_value > current_value:
    print('BUY')
else:
    print('SELL')

SELL


## Enterprise Valiation - alternative

In [92]:
# This is another way of calculating PV of FFCF - sum of it should match the npv calculation of npf
pv_ffcf = []
for idx, cf in enumerate(ffcf):
    pv_ffcf.append(cf/(1 + WACC_RATE)**(idx+1))
# This is equivalent to enterprise_value value
sum(pv_ffcf)

1950361999312.4448